In [9]:
from __future__ import division
import shutil as sh
from os.path import join 
from glob import glob
import itertools
from distutils.dir_util import copy_tree
import shutil as shutil
from glob import glob
import pandas as pd

import numpy as np
from subprocess import check_call
import matplotlib.pyplot as plt
#from utilities import ensure_dir, get_n_vols
from pylab import plot, ylim, xlim, show, xlabel, ylabel, grid
from numpy import linspace, loadtxt, ones, convolve
from nipype.interfaces.dcm2nii import Dcm2niix, Dcm2nii
#import nipype_init_new
#nipype_init_new.init()
from nilearn.connectome import sym_matrix_to_vec, vec_to_sym_matrix,ConnectivityMeasure
from nilearn.plotting import plot_stat_map, plot_anat, plot_img, view_img, plot_matrix, plot_glass_brain
from nilearn.image import concat_imgs, mean_img, load_img, index_img, math_img, resample_img
from nilearn.masking import apply_mask
from nilearn.plotting import plot_stat_map, plot_anat, plot_img, view_img
from nilearn import plotting
from statsmodels.sandbox.stats.multicomp import multipletests as multi
from scipy.stats import norm
import statsmodels.stats.multitest as ssm
import statsmodels
import seaborn as sns
from scipy.stats import pearsonr,ttest_ind
from sklearn.model_selection import StratifiedKFold,permutation_test_score
from sklearn.svm import SVC
from sklearn.model_selection import LeaveOneOut
from sklearn.svm import SVR
import time

In [10]:
# migrate functions in nistats to nilearn
# https://nilearn.github.io/nistats_migration.html#design-matrix
from nilearn.glm.first_level import FirstLevelModel
#from nistats.thresholding import map_threshold # can not be migrate directly
from nilearn.glm import threshold_stats_img
from nilearn.reporting import get_clusters_table
from nilearn.plotting import plot_contrast_matrix, plot_design_matrix
from nilearn.glm.first_level import make_first_level_design_matrix
from nilearn.glm.second_level import SecondLevelModel
from nltools.data import Brain_Data

In [11]:
import warnings
warnings.filterwarnings("ignore")

In [12]:
def ensure_dir(path):
    d = os.path.dirname(path)
    if not os.path.exists(d):
        os.makedirs(d)


In [13]:
topdir='/home/haiyanwu/nas_data/experiment_data/uarep_only/'
roi_dir=topdir+'3.ROIextraction/'
mask_dir=roi_dir+'masks/'



# distinctiveness

In [14]:
ROIs=['BA9-dlPFC','BA24-dACC','BA34-hip','lDLPFC','rDLPFC','ventral_striatum','putamen','rTPJ','lTPJ','BA21-MTG','BA23-PCC','BA47-vlPFC','mPFC',
        'NAcc','ACC','BA33-rACC','amyg','amygL','amygR','precuneus','caudate','BA45-IFG','BA10-FPC']
len(ROIs)

35

In [15]:
subList=[101,102,103,105,106,107,108,109,110,111,112,115,116,117,118,119,120,121,123,125,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143]
len(subList)

37

In [18]:

import os
glmdir=topdir+'1.GLM/results/GLM7uarep_LSS_unsmooth/betamap/'
writer = pd.ExcelWriter('ROIdistinctiveness_elie_rep.xlsx', engine='xlsxwriter')
#for indroi, roi in enumerate(ROIs):
for indroi, roi in enumerate(ROIs):
    mask=Brain_Data(mask_dir+'Mask_%s.nii'%(roi))
    
    Dists=[]
    for indsub, subno in enumerate(subList):
        
        subdist=[]
        betalists=os.listdir(glmdir+'sub-%03d'%(subno))
        for runno in [1,2,3,4,5,6,7,8]:
            trialdata=Brain_Data()
            for menuno in range(1,13):
                fname=glmdir+'sub-%03d/sub-%03d_run%01d_menu%01d_eff_map.nii.gz'%(subno,subno,runno,menuno*3-1)

          
                trialdata=trialdata.append(Brain_Data(fname))
            
            
                
            dist = sum(trialdata.apply_mask(mask).distance(metric='correlation').data)/j/(j-1)
            subdist.append(dist)
        Dists.append(subdist)
    Dists=pd.DataFrame(Dists,index=subList)
    Dists.to_excel(writer, index=subList,header=[1,2,3,4,5,6,7,8], sheet_name='%s' % roi, startcol=1)
           
writer.save()   